# Prototype 1b, tests on pdf 2
**Stack:**
```
OCR: Gemini 2.5 flash
Embeddings: gemini-embedding-exp-03-07
LLM: Gemini 2.5 flash

```
It requires credentials, you will need to download googles authentication CLI. Check their [official documentation](https://cloud.google.com/vision/docs/authentication)

> Installation

In [2]:
%pip install --upgrade google-genai
%pip install python-dotenv
%pip install requests
%pip install faiss 


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
ERROR: Could not find a version that satisfies the requirement faiss (from versions: none)
ERROR: No matching distribution found for faiss
Note: you may need to restart the kernel to use updated packages.


In [3]:
#Variables
from dotenv import load_dotenv
load_dotenv()
import os
OCR_API_KEY=os.environ.get("GCV_API_KEY")
EMBEDDINGS_API_KEY=os.environ.get("GEMINI_API_KEY")
MILO_API_KEY=os.environ.get("MILO_API_KEY")
doc_path="Note_clinique_chambre_implantable_2.pdf"
questionnaire_id = 1182 # Update this to the correct questionnaire ID

> OCR

In [4]:
import os
from google import genai
from google.genai.types import (
    GenerateContentConfig,
    HarmBlockThreshold,
    HarmCategory,
    HttpOptions,
    Part,
    SafetySetting,
)


client = genai.Client(http_options=HttpOptions(api_version="v1"), api_key=OCR_API_KEY)


# Read the PDF file as bytes
with open(doc_path, "rb") as pdf_file:
    pdf_bytes = pdf_file.read()

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=[
        Part.from_bytes(
            data=pdf_bytes,
            mime_type="application/pdf",
        ),
        """Extract and output the positions of objects and the text in them for each page. Return the output in JSON format without markdown fomatting. Like this:
            [
                {
        "page": 1,
        "objects": [
        {
            "box_2d": [75, 48, 89, 401],
            "text": "  "
        }
        ]
            }
        ]
        
        """,
    ],
)

> Prepare the output for embeddings. 

In [5]:
import json
response_json=json.loads(response.text)
all_text = " ".join(
    obj.get("text", obj.get("text_content", "")) 
    for page in response_json 
    for obj in page["objects"]
)
all_words=all_text.split()
chunk_size=50
split_size=len(all_words) //chunk_size
embed_input=[]
for i in range(split_size):
    embed_input.append(" ".join(all_words[i*chunk_size:(i+1)*chunk_size]))
embed_input.append(" ".join(all_words[split_size*chunk_size:]))
print(embed_input)


["Nom du patient : Sarah Thompson Date de naissance : 22/03/1990 Âge : 35 ans Taille : 1,63m Poids : 65kg Sexe : Féminin Numéro de dossier médical (NDM) : 987654321 Date de l'intervention : 15/06/2025 Heure : 14h45 à 15h20 Chirurgien responsable : Dr Amanda Lee, MD, PhD Assistant(e)", ": Jordan Kim, IDE Procédure : Pose d'une chambre d'implantation sous-cutanée crânienne Localisation : Région pariétale gauche Anesthésie : e (Sevoflurane + Propofol IV) Allergies : Aucune allergie médicamenteuse connue (AAMK) --- Note opératoire La patiente a subi avec succès la pose d'une chambre d'implantation crânienne 38L x25-43 à visée", "expérimentale, permettant un accès chronique à la surface corticale. L'intervention s'est déroulée sans complication. --- S - Subjectif Patiente de 35 ans, sans antécédents médicaux significatifs, incluse dans un protocole de recherche en neurosciences. Elle consulte pour la mise en place d'une chambre crânienne afin de permettre un accès cortical", 'répété. Elle n

> Getting the survey

In [6]:
import requests

def get_questionnaire_data(questionnaire_id):
 
    # Replace with your API key
    api_key = MILO_API_KEY
    
    # Replace with the URL you want to access
    url = f"https://dev.milo-dct-backend.com:8443/api/v1/questionnaires/{questionnaire_id}/questions-ai"
 
    # Headers to include the API key
    headers = {
        "Authorization": f"Bearer {api_key}"
    }
 
    # Make the GET request
    
    response = requests.get(url, headers=headers)
 
    # Check if the request was successful
    if response.status_code == 200:
        return response.json()  # or response.text
    else:
        print(f"Request failed with status code : {response.status_code}")
        return response.text
    
data = (get_questionnaire_data(questionnaire_id))
print(data)

# Extract question headers
question_titles = [question["questionTitle"] for question in data]
print(question_titles)


[{'id': 6920, 'questionTitle': 'Sex du patient', 'questionType': 'DROPDOWN', 'options': [{'id': 44594, 'label': 'Male'}, {'id': 44595, 'label': 'Female'}]}, {'id': 6921, 'questionTitle': 'Taille (en cm)', 'questionType': 'DECIMAL'}, {'id': 6922, 'questionTitle': 'Poids (en Kg)', 'questionType': 'DECIMAL'}, {'id': 6923, 'questionTitle': 'Date de naissance', 'questionType': 'YEAR_AND_MONTH'}]
['Sex du patient', 'Taille (en cm)', 'Poids (en Kg)', 'Date de naissance']


> Embedding

In [ ]:
from google import genai
from google.genai import types
client = genai.Client(api_key=EMBEDDINGS_API_KEY)
embedded_text = client.models.embed_content(
        model="gemini-embedding-exp-03-07",
        contents=embed_input,
        config=types.EmbedContentConfig(task_type="SEMANTIC_SIMILARITY")
)
embedded_questions = client.models.embed_content(
        model="gemini-embedding-exp-03-07",
        contents=question_titles,
        config=types.EmbedContentConfig(task_type="SEMANTIC_SIMILARITY")
)


[ContentEmbedding(
  values=[
    -0.025878742,
    0.0020673582,
    0.013390861,
    -0.059618957,
    -0.011064952,
    <... 3067 more items ...>,
  ]
), ContentEmbedding(
  values=[
    -0.007082646,
    0.0068139415,
    -0.001481578,
    -0.06914881,
    -0.008830571,
    <... 3067 more items ...>,
  ]
), ContentEmbedding(
  values=[
    -0.019181354,
    0.015095852,
    -0.0021383704,
    -0.054811887,
    -0.013169268,
    <... 3067 more items ...>,
  ]
), ContentEmbedding(
  values=[
    -0.009992672,
    -0.001156016,
    0.0018930413,
    -0.06021864,
    -0.009157293,
    <... 3067 more items ...>,
  ]
), ContentEmbedding(
  values=[
    0.010657949,
    0.010105528,
    -0.00038620213,
    -0.06531801,
    -0.013499003,
    <... 3067 more items ...>,
  ]
), ContentEmbedding(
  values=[
    -0.007613509,
    0.002130284,
    0.011550773,
    -0.07004851,
    -0.0072456836,
    <... 3067 more items ...>,
  ]
), ContentEmbedding(
  values=[
    -0.0044787005,
    -0.01132994

> Performing a similarity search with the faiss library.

In [8]:
import numpy as np
import faiss

# Convert Gemini embeddings to numpy arrays
answer_embeddings = np.array([e.values for e in embedded_text.embeddings], dtype=np.float32)
question_embeddings = np.array([e.values for e in embedded_questions.embeddings], dtype=np.float32)

dimension = answer_embeddings.shape[1]

# Normalize for cosine similarity
faiss.normalize_L2(answer_embeddings)
faiss.normalize_L2(question_embeddings)

# Create the FAISS index for cosine similarity
index = faiss.IndexFlatIP(dimension)
index.add(answer_embeddings)

# Search for the nearest neighbors
k = 3
D, I = index.search(question_embeddings, k)


> Prepare the text and parse it for the LLM

In [34]:
subQuestions = [question.get("options", []) for question in data]
all_answers = {}
for i in range(len(subQuestions)):
    # Combine main and subquestions into a flat list
    LLMquestions = [question_titles[i]] + [q["label"] for q in subQuestions[i]]
    questions_str = "\n".join(f"- {q}" for q in LLMquestions)

    # Build context string
    LLMcontext = []
    for j in range(k):
        idx = I[i][j]
        if idx < len(embed_input):
            LLMcontext.append(embed_input[idx])
    context_str = "\n".join(LLMcontext)

    # Generate response
    response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=[
        f"""
    Answer the following questions using only the information from the context.
    Return your answers as a single JSON object without markdown quotes, with each question as a key and the answer as the value.
    If an answer is not found in the context, use "Not found in context." as the value.
    DO NOT CHANGE THE FORMAT. ex:1,63m is not changed to 163cm
    Questions:
    {questions_str}

    Context:
    {context_str}
    """
        ],
    )
        #  Parse the JSON response
    try:
        answer_dict = json.loads(response.text)
    except json.JSONDecodeError:
        print("Warning: Could not parse response as JSON:", response.text)
        continue

    # Update the cumulative dictionary
    all_answers.update(answer_dict)

# Now all_answers contains all the merged results
print(all_answers)


{'Sex du patient': 'Féminin', 'Male': 'Not found in context.', 'Female': 'Féminin', 'Taille (en cm)': '1,63m', 'Poids (en Kg)': '65kg', 'Date de naissance': '22/03/1990'}


> Read the output with json and put the values in a list for highlighting.

In [ ]:

import json
searchValues=set(list(all_answers.values())) # Here we store the values in a list, and remove the repeated ones. 
#we get back the list from the beggining response_json


In [45]:
# Find bounding boxes for each value
results = []
for page in response_json:
    page_num = page.get("page", None)
    for obj in page["objects"]:
        for value in searchValues:
            if value in obj["text"]:
                results.append({
                    "value": value,
                    "box_2d": obj["box_2d"],
                    "text": obj["text"],
                    "page": page_num
                })



> Now we highlight the pdf at the relevant places.

In [50]:
import fitz
# === Open the PDF ===
doc = fitz.open(doc_path)
output_path=doc_path+"highlighted.pdf"
# === Highlight each box on the correct page ===

# Convert Gemini coordinates to actual PDF coordinates
def convert_gemini_to_pdf(box_2d, pdf_width, pdf_height):
    # Gemini format: [top, left, bottom, right] in 1000x1000 scale
    # Convert to actual image coordinates first
    top = (box_2d[0] / 1000) * pdf_height
    left = (box_2d[1] / 1000) * pdf_width  
    bottom = (box_2d[2] / 1000) * pdf_height
    right = (box_2d[3] / 1000) * pdf_width
    
    # Return in PyMuPDF format: [left, top, right, bottom]
    return [left, top, right, bottom]


for result in results:
    page_num = result["page"] - 1  # PyMuPDF uses 0-based page numbers
    box = result["box_2d"]
    # Get page dimensions
    pdf_width = page.rect.width
    pdf_height = page.rect.height
    # If box = [top, left, bottom, right], convert to [left, top, right, bottom]
    x0, y0, x1, y1 =convert_gemini_to_pdf(box,pdf_width,pdf_height) 
    rect = fitz.Rect(x0, y0, x1, y1)
    page = doc[page_num]
    annot = page.add_rect_annot(rect)
    annot.set_colors(stroke=(1, 1, 0), fill=(1, 1, 0))  # Yellow
    annot.set_opacity(0.4)  # Semi-transparent
    annot.update()


# === Save the highlighted PDF ===
doc.save(output_path)